In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import json
import os
import os
import json


def remove_underline(x):
    return x.replace("_", "")

In [2]:
task_to_label_spaces = {'amazon': ['positive', 'negative'], 'sst2': ['positive', 'negative'], 'yelp': ['positive', 'negative'],
                        'tweet_hate': ['Hate', 'Non-hate',], 'tweet_offensive': ['Hate', 'Non-hate'],
                        'agnews': {'world', 'science', 'sports', 'business'},
                        'dbpedia': {"Company", "EducationalInstitution", "Artist", "Athlete", "OfficeHolder",
                                    "MeanOfTransportation", "Building", "NaturalPlace", "Village", "Animal", "Plant",
                                    "Album", "Film", "WrittenWork"}
                        }

In [3]:
def normalise_pred(pred):
    return pred.strip().split("\n")[0].strip()

def check_file_in_label_rate(pred_data, task):
    assert len(pred_data['prompts']) == len(pred_data['preds'])
    invalid_labels = [x for x in pred_data['preds'] if normalise_pred(x) not in task_to_label_spaces[task]]
    return 1 - len(invalid_labels) / len(pred_data['preds']), invalid_labels

In [4]:
BASE_PATH = '/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/'
commands = []
data = []


high_priority=False

# size="360M"
# iter_name = "iter-160000-ckpt-step-40000_hf"
#iter_name = "iter-110000-ckpt-step-27500_hf"
# iter_name = "iter-260000-ckpt-step-65000_hf"

size="1b"
# iter_name = "iter-480000-ckpt-step-60000_hf"
# iter_name = "iter-380000-ckpt-step-47500_hf"
iter_name = "iter-600000-ckpt-step-75000_hf"
# model_names = ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v2_8k_intracccont','cc_8k', 'cc_merged_v1_8k', 'adamask_cc_merged_v2_8k', ] # 'intramask_cc_merged_v2_8k']
# model_names = ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v2_8k_intracccont','cc_8k',   'adamask_cc_merged_v2_8k', 'intramask_cc_merged_v2_8k']
# model_names = ['intramask_cc_8k', 'intramask_cc_merged_v2_8k', 'cc_merged_v2_8k_intrav2cont','cc_8k', 'adamask_cc_merged_v2_8k',  'cc_merged_v2_8k',]
model_names = ['cc_merged_v2_8k']
# model_names = ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v1_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intracccont','cc_8k',]
# model_names = ['baseline']


# TASK_CLASS, shot_nums, rag_num_docs_lst ="obqa", [8, 16], [""]
# TASK_CLASS, shot_nums, rag_num_docs_lst ="obqa", [4], [""]
TASK_CLASS, shot_nums, rag_num_docs_lst ="icl", [24,48], [""]
# TASK_CLASS, shot_nums, rag_num_docs_lst ="cbqa", [12,24], [""]
# TASK_CLASS, shot_nums, rag_num_docs_lst ="obqa_rag", [3], [1,3,5,10]


class_to_tasks_mapping = {"icl": ['agnews' ,'amazon' ,'dbpedia' ,'sst2','tweet_hate' ,'tweet_offensive' ,'yelp' ],
                          "obqa": [ "squad","hotpotqa"],
                          "obqa_rag": ["nq_obqa", "tq_obqa"],
                          "cbqa": ["tq", "nq"],
                          }

class_to_seed_mapping = {"icl": range(42, 58),
                         "obqa": range(42, 46+1),
                         "cbqa": range(42, 46 + 1),
                            "obqa_rag": range(42, 46 + 1),
                         }
for task_name in  class_to_tasks_mapping[TASK_CLASS]:
    for shot_num in shot_nums:
        for rag_num_docs in rag_num_docs_lst:
            if task_name == "yelp" and shot_num == 48:
               continue
            for model_name in model_names:
                if '8k' not in model_name:
                    model_name = f"{model_name}_8k"
                for seed_num in class_to_seed_mapping[TASK_CLASS]:
                    full_name = f"tiny_LLaMA_{size}_8k_{model_name}-{iter_name}"
                    # full_name = "TinyLlama-1.1B-step-50K-105b"
                    if TASK_CLASS == "obqa_rag":
                        result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}_{rag_num_docs}.json')
                    else:
                        result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}.json')
                    if not os.path.exists(result_path):
                        continue
                    else:
                        print("Existing", result_path)
                        acc_result = json.load(open(result_path, 'r'))
                        pred_path = result_path.replace(".json", '_prompts_and_preds.json')
                        pred_result = json.load(open(pred_path, 'r'))
                        acc_result['valid_ratio'], invalid_preds = check_file_in_label_rate(pred_result, task_name)
                        # assert (acc_result['valid_ratio'] > 0.9) or task_name in ['dbpedia', 'agnews'] , f"valid_ratio is {acc_result['valid_ratio']}"
                        if acc_result['valid_ratio'] < 0.9:
                            if task_name not in ['dbpedia', 'agnews']:
                                print("Task", task_name, "Sample invalid preds", invalid_preds[:10])
                        if TASK_CLASS == "obqa_rag":
                            acc_result['num_docs'] = rag_num_docs
                        data.append(acc_result)



Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-ckpt-step-75000_hf/agnews_24_42.json
Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-ckpt-step-75000_hf/agnews_24_43.json
Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-ckpt-step-75000_hf/agnews_24_44.json
Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-ckpt-step-75000_hf/agnews_24_45.json
Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-ckpt-step-75000_hf/agnews_24_46.json
Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-ckpt-step-75000_hf/agnews_24_47.json
Existing /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/tiny_LLaMA_1b_8k_cc_merged_v2_8k-iter-600000-c

In [5]:
df = pd.DataFrame(data)

In [6]:
df

,task,n_shot,seed,model,score,valid_ratio
0,agnews,24,42,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,72.289474,0.982763
1,agnews,24,43,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,68.421053,0.982632
2,agnews,24,44,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,63.618421,0.984474
3,agnews,24,45,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,67.157895,0.974211
4,agnews,24,46,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,78.315789,0.976579
...,...,...,...,...,...,...
189,yelp,24,53,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,83.860000,0.991200
190,yelp,24,54,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,91.340000,0.990200
191,yelp,24,55,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,91.540000,0.997800
192,yelp,24,56,tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-60...,92.540000,0.992800
